In [13]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda,RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings


In [9]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() 
client = OpenAI()

In [30]:
agente_rotas_template = """Você é um especialista no sistema de Metrô de São Paulo, com foco em determinar as melhores rotas para os passageiros. \
      Você tem amplo conhecimento sobre as linhas do metrô, suas interligações e horários de pico, fornecendo alternativas rápidas e eficientes para quem utiliza o metrô.

Aqui está a pergunta:
{query}"""

agente_horarios_template = """Você é um especialista em atualizações sobre o sistema de Metrô de São Paulo. \
    Você informa sobre greves, interrupções e atrasos, além de conhecer detalhadamente os horários de operação do metrô. \
        Você sugere opções alternativas para os passageiros durante períodos de interrupção do serviço.

Aqui está a pergunta:
{query}"""

agente_politicas_template = """Você é um especialista em políticas públicas relacionadas ao sistema de Metrô de São Paulo. \
    Você tem conhecimento detalhado sobre as leis, regulamentos e iniciativas que afetam o metrô, e oferece respostas claras \
        e baseadas em fatos sobre como essas políticas impactam os passageiros.

Aqui está a pergunta:
{query}"""



In [12]:
rotas_perguntas = [
    "Qual é a rota mais rápida da estação Sé até a estação Vila Madalena?",
    "Como posso evitar as estações mais lotadas durante o horário de pico?",
    "Quais são as melhores conexões entre as linhas azul e verde do metrô?",
]

horarios_perguntas = [
    "Há alguma greve no Metrô de São Paulo programada para esta semana?",
    "O metrô está funcionando normalmente durante o feriado?",
    "Quais são os horários de operação do metrô aos domingos?",
]

politicas_perguntas = [
    "Quais são as políticas de acessibilidade nas estações do Metrô de São Paulo?",
    "Como as tarifas do metrô são decididas e ajustadas?",
    "Quais são as políticas de sustentabilidade adotadas pelo Metrô de São Paulo?",
]


In [31]:
embeddings = OpenAIEmbeddings()

rotas_perguntas_embeddings = embeddings.embed_documents(rotas_perguntas)
horarios_perguntas_embeddings = embeddings.embed_documents(horarios_perguntas)
politicas_perguntas_embeddings = embeddings.embed_documents(politicas_perguntas)

In [35]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])

    rotas_similarity = cosine_similarity([query_embedding], rotas_perguntas_embeddings)[0]

    horarios_similarity = cosine_similarity([query_embedding], horarios_perguntas_embeddings)[0]

    politicas_similarity = cosine_similarity([query_embedding], politicas_perguntas_embeddings)[0]

    max_similarity = max(
        max(rotas_similarity), max(horarios_similarity), max(politicas_similarity)
    )

    if max_similarity == max(rotas_similarity):
        print("rotas")
        return PromptTemplate.from_template(agente_rotas_template)
    elif max_similarity == max(horarios_similarity):
        print("horarios")
        return PromptTemplate.from_template(agente_horarios_template)
    else:
        print("politicas")
        return PromptTemplate.from_template(agente_politicas_template)


input_query = {"query": "Quando o metrô abre e fecha nos finds?"}
prompt = prompt_router(input_query)

horarios


In [36]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [37]:
chain.invoke("Como vou da fradique coutinho ate a sacoma?")

rotas


'Para ir da estação Fradique Coutinho até a estação Sacomã, você pode seguir a seguinte rota:\n\n1. Pegue a Linha 4-Amarela do metrô na estação Fradique Coutinho e siga até a estação Paulista.\n2. Na estação Paulista, faça a transferência para a Linha 2-Verde e siga até a estação Paraíso.\n3. Na estação Paraíso, faça a transferência para a Linha 1-Azul e siga até a estação Ana Rosa.\n4. Na estação Ana Rosa, faça a transferência para a Linha 2-Verde novamente e siga até a estação Sacomã.\n\nEssa rota pode variar de acordo com os horários de pico e eventuais interrupções no sistema. Sempre verifique a situação do metrô antes de sair, para garantir uma viagem tranquila e eficiente.'